In [ ]:
from datetime import datetime

import numpy as np
import torch
import torch.nn as nn

from utils_io import read_bitarrays
from utils_attacker_lstm import DatasetAttackerLSTMPool, DataLoaderAttackerLSTM, ModelAttackerConvLSTMLinear, LSTMAttackerTrainer, LSTMAttackerTester
from utils_torch import stratified_random_split

In [ ]:
num_snps = 40000
genomes_pool = read_bitarrays('../data/test/In_Pop.pkl')[:, :num_snps]
genomes_reference = read_bitarrays('../data/test/Not_In_Pop.pkl')[:, :num_snps]
genomes = np.concatenate((genomes_pool, genomes_reference), axis=0)

In [ ]:
labels_beacon = np.ones(genomes_pool.shape[0], dtype=bool)
labels_reference = np.zeros(genomes_reference.shape[0], dtype=bool)
labels = np.concatenate((labels_beacon, labels_reference), axis=0).astype(bool)

In [ ]:
frequencies_pool = np.mean(genomes_pool, axis=0)
frequencies_reference = np.mean(genomes_reference, axis=0)

In [ ]:
dataset = DatasetAttackerLSTMPool(
    target_genomes=genomes,
    pool_frequencies=frequencies_pool,
    reference_frequencies=frequencies_reference,
    labels=labels)
subset_train, subset_eval, subset_test = stratified_random_split(dataset, [0.7, 0.15, 0.15])

In [ ]:
genomes_batch_size, snps_batch_size = 32, 20000
loader_train = DataLoaderAttackerLSTM(subset_train, genomes_batch_size, snps_batch_size, shuffle=True)
loader_eval = DataLoaderAttackerLSTM(subset_eval, genomes_batch_size, snps_batch_size, shuffle=False)
loader_test = DataLoaderAttackerLSTM(subset_test, genomes_batch_size, snps_batch_size, shuffle=False)

In [ ]:
if torch.backends.mps.is_available():
    device = torch.device('mps')
elif torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [ ]:
model = ModelAttackerConvLSTMLinear(
    
)